#README.txt

Drag and drop a ZIPPED version of the folder `/data` from GitHub repo into Colab.

< - - - - -

In [ ]:
# then run this and refresh directory...

# unzip datasets
!unzip data.zip



---


# API

In [1]:
!pip install config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
import os
import config
# API Docs found here: https://developer.usajobs.gov/Tutorials/Search-Jobs

In [3]:
config.US_JOBS_API_KEY = "xVc0TZiLfhcr17ci7Ngk6bLAetdRVFgntm2pZgWNtww="
config.EMAIL_ADDRESS = "gjacobthomas@gmail.com"

Note: I'm assuming this API has some limiter on it so we don't want to lose access. -tyler

In [4]:
# more parameters are found here: https://developer.usajobs.gov/API-Reference/GET-api-Search
def queryJobsAPI(keyword, location):
  host = 'data.usajobs.gov' 
  # add these values in the config.py file
  userAgent = config.EMAIL_ADDRESS
  authKey = config.US_JOBS_API_KEY

  base_url = "https://data.usajobs.gov/api/search"

  parameters = {
      "Keyword": keyword,
      "LocationName": location
  }

  headers = {
      "Host": host,          
      "User-Agent": userAgent,          
      "Authorization-Key": authKey  
  }

  resp = requests.request("GET", base_url,headers=headers, params=parameters)
  result = resp.json()['SearchResult']['SearchResultItems']

  # get Job Title 
  print(result[1]['MatchedObjectDescriptor']['PositionTitle'])
  # get Job Summary
  print(result[1]['MatchedObjectDescriptor']['UserArea']['Details']['JobSummary'])
  return result




---


# KeyBERT Extraction Function

In [ ]:
!pip install keybert

In [6]:
# Imports
from keybert import KeyBERT # pip install keybert (give it a minute...)
from statistics import mean

'''
/*---------------------------------------------------------------------
 |  Method: extractKeywordsBERT
 |
 |  Purpose: Uses the KeyBert Keyword Extraction Tool to extract
 |           and return keywords from a given corpus. 
 |      
 |  Author: Tyler Parks
 |  Created On: 10/30/22
 |
 |  Parameters:
 |      normalized_corpus -- A single string containing all text of the
 |                           normalized corpus.
 |
 |  Returns: 
 |      keywords -- List of collected keywords
 |      scores -- List of those keyword's scores
 |
 |  References: https://maartengr.github.io/KeyBERT/#usage
 |
 *-------------------------------------------------------------------*/
''' 
def extractKeywordsBERT(normalized_corpus):   
    print('---KeyBert Extraction---')
    print('------------------------\n')

    # init. language model 
    language_model = KeyBERT(model = 'all-mpnet-base-v2')

    # extract those keywords!
    data = language_model.extract_keywords( normalized_corpus, 
                                            keyphrase_ngram_range=(1, 3), 
                                            stop_words='english',
                                            use_maxsum=False, 
                                            use_mmr=True,
                                            diversity=0.7,
                                            nr_candidates=20, 
                                            top_n=15
                                        )

    # zip the lists
    zipped = list(map(list, zip(*data)))
    keywords = zipped[0]
    scores = zipped[1]

    print('-Skill-'.ljust(40), '-Score-')
    for i, value in enumerate(keywords):
        print(value.ljust(40), scores[i])
    print()

    # print("type: "+str(type(scores))+"length: "+ str(len(scores)))

    avg = mean(scores[:14])
    print("Score Average: " + str(avg))

    return keywords, scores, avg



---


# CSO-Classifier Extraction Function

In [7]:
!python -m spacy download en_core_web_sm
!pip install cso-classifier

2022-12-03 18:33:41.596750: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 4.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-3.0.5-cp38-cp38-manylinux2014_x86_64.whl (12.9 MB)
  Using cached thinc-8.0.17-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (671 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.5
    Uninstalling thinc-8.1.5:
      Successfully uninstalled thinc-8.1.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.3
    Uninstalling spacy-3.4.3:
      Successfully uninstalled spacy-3.4.3
ERROR: pip's depend

**Don't restart runtime if the terminal says so! Keep going.**

In [8]:
import nltk
nltk.download('stopwords')
# update spacy
!pip install -U spacy
import spacy
from cso_classifier import CSOClassifier      # import classifier tool

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-3.4.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Using cached thinc-8.1.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (819 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.5
    Uninstalling spacy-3.0.5:
      Successfully uninstalled spacy-3.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cso-classifier 3.0 requires spacy==3.0.5, but you have spacy 3.4.3 which is incompatible.


In [9]:
# update the most recent model
CSOClassifier.update() 

# define the model object
CSO_Extractor = CSOClassifier(modules = "both", enhancement = "first", explanation = False)


# ======================================================
#     ONTOLOGY
# ======================================================
The ontology is already up to date.

# ======================================================
#     MODELS: CACHED & WORD2VEC
# ======================================================
Updating the models: cached and word2vec
[██████████████████████████████████████████████████] 63M/63M
[*] Done!
[██████████████████████████████████████████████████] 349M/349M
[*] Done!
Models downloaded successfully.
Update completed.


In [58]:
def extractKeywordsCSO(normalized_corpus):

  # run the extraction
  result = CSO_Extractor.run(normalized_corpus)

  print('\n-----CSO Extraction-----')
  print('------------------------\n')
  
  for keyword in result['union']:
    print(keyword)

  return result['union']

# **YAKE Extractor**

In [11]:
!pip3 install yake
import yake 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 3.2 MB/s 
     |████████████████████████████████| 132 kB 10.1 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp38-cp38-linux_x86_64.whl size=70607 sha256=f7060ab76a2d6a5febb3f6fc00478b0ddad5ea0b41afe253d5fda14da1aa0df9
  Stored in directory: /root/.cache/pip/wheels/f1/c7/3c/4c83132de76359e3a429fd09c08995945ca96c5290a41651d3
Successfully built jellyfish


In [71]:
from statistics import mean

def extractKeywordsYAKE(normalized_corpus):
    print('\n---YAKE Extraction---')
    print('------------------------\n')

    language = "en"
    max_ngram_size = 3
    deduplication_threshold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 20

    kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = kw_extractor.extract_keywords(normalized_corpus)

    zipped = list(map(list, zip(*keywords)))
    keywords = zipped[0]
    scores = zipped[1]

   
    for i, value in enumerate(keywords):
        print(value.ljust(40), scores[i])

    avg = mean(scores[:19])
    print("Score Average: " + str(avg)+ "\n")


    return keywords, scores, avg

# **TextRank Extractor**

In [13]:
!pip install summa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.0 MB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54410 sha256=a5519050f640a090b82ae91d495b9d670914a9d4a60610fa84d60be429999f74
  Stored in directory: /root/.cache/pip/wheels/fd/6a/dd/209eb19d5f2266b9cfd06827539bf70435b0ad5fe8244e52d3
Successfully built summa


In [14]:
from summa import keywords
from statistics import mean

def extractKeywordsTextRank(normalized_corpus):
  print('\n')
  print('---TextRank Extraction---')
  print('------------------------\n')

  #extract
  TR_keywords = keywords.keywords(normalized_corpus, scores=True)

  #zip into list
  zipped = list(map(list, zip(*TR_keywords)))
  TR_keywords = zipped[0]
  scores = zipped[1]
  

  #print(TR_keywords[0:20])
  print('-Skill-'.ljust(40), '-Score-')
  for i, value in enumerate(TR_keywords[0:19]):
    print(value.ljust(40), scores[i])
    
  avg = mean(scores[:19])
  print("\nScore Average: " + str(avg))
  
  

  return TR_keywords, scores, avg
  
  

# **Skill Matching**

In [15]:
!pip install gensim
!pip install jieba

from functools import reduce
import gensim
import nltk
from nltk.tokenize import word_tokenize
import jieba
nltk.download('punkt')
import re
import numpy as np

class SkillsMatching:
  def __init__(self, all_job_postings_keywords, all_jp_titles, all_resumes_keywords):
      self.all_job_postings_keywords = all_job_postings_keywords
      self.all_jp_titles = all_jp_titles
      self.all_resumes_keywords = all_resumes_keywords
      

  # output JSON with matching job postings for each resume
  def skill_match(self):
      results = []
      
      for resume_json in self.all_resumes_keywords:
          resume_arr = []
          for key in resume_json:
            # loop through the keys = every classifier 
            classifier_json = {key: {"matching_job": "",
                      "job_title": ""}}
            resume_keywords = resume_json[key]
            scores = self.sim_score(self.all_job_postings_keywords, resume_keywords)
            # grab the index of the largest score in the arr
            max_idx = np.argmax(scores)
            classifier_json[key]['matching_job'] = self.all_job_postings_keywords[max_idx]
            classifier_json[key]['job_title'] = self.all_jp_titles[max_idx]
            resume_arr.append(classifier_json)
          results.append(resume_arr)
      return results


  def split_and_join_arr(self, arr):
    new_arr = []
    for w in arr:
      word_arr = re.split('\W+', w.lower())
      new_arr = new_arr + word_arr
    # print(new_arr)
    return new_arr
  
  def sim_score(self, docs, keywords):
    keywords = self.split_and_join_arr(keywords)
    gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in docs]
    dictionary = gensim.corpora.Dictionary(gen_docs)
    corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
    tf_idf = gensim.models.TfidfModel(corpus)
    sims = gensim.similarities.Similarity('/usr/workdir',tf_idf[corpus],
                                      num_features=len(dictionary))

    query_doc_bow = dictionary.doc2bow(keywords)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    # print(sims[query_doc_tf_idf])
    return sims[query_doc_tf_idf]

  def print_results(self, match_job_postings):
    for i, resume in enumerate(match_job_postings):
      print('------------------------\n')
      print("Matching Job Posting for Resume %s" % (i + 1))
      for classifier_obj in resume:
        classifier_type = list(classifier_obj.keys())[0]
        print(classifier_type)
        print(classifier_obj[classifier_type]['job_title'])
        print(classifier_obj[classifier_type]['matching_job'])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.




---


#Driver Code

In [ ]:
!pip install contractions

In [ ]:
# Imports
import pandas as pd     # pip install pandas. usage: loading data from csv files into dataframes
import nltk
from nltk.tokenize import word_tokenize  
from nltk.stem import WordNetLemmatizer
import re
import contractions
import spacy
from numpy.lib.npyio import savez_compressed
from array import *
nltk.download("all")
nltk.download('stopwords')
from nltk.corpus import stopwords
nlp = spacy.load('en_core_web_sm')

In [90]:
### Helper Functions

# Function to retrieve text data.
# (either 1 or more job postings or resumes)?
def getFileData(filename, dir):
    return pd.read_csv('data/' + dir + '/' + filename)

# Function to normalize text data. 
# (some skill extraction tools will normalize text for us; however, if not, this function is here)
# includes removing stopwords, punctuation, dates, links, etc...
def normalizeCorpus(corpus):
    
    # import nltk for stopwords

    stop_words = set(stopwords.words('english'))

    # convert to lower case
    lower_corpus = corpus.lower()

    # remove numbers
    no_number_corpus = re.sub(r'\d+','',lower_corpus)

    # remove all punctuation except words and space
    no_punc_corpus = re.sub(r'[^\w\s]','', no_number_corpus)

    # remove white spaces
    no_wspace_corpus = no_punc_corpus.strip()
    no_wspace_corpus

    # convert string to list of words
    lst_corpus = [no_wspace_corpus][0].split()
    print(lst_corpus)

  # remove stopwords
    no_stpwords_corpus=""
    for i in lst_corpus:
	    if not i in stop_words:
	      no_stpwords_corpus += i+' '
		
    # removing last space
    no_stpwords_corpus = no_stpwords_corpus[:-1]

    # output
    print(no_stpwords_corpus)

    return no_stpwords_corpus
    # return corpus
    # need to fix this 
    # nltk_tokenList = word_tokenize(corpus)
    # lemmatizer = WordNetLemmatizer()
    # nltk_lemmaList = []
    # for word in nltk_tokenList:
    #     nltk_lemmaList.append(lemmatizer.lemmatize(word))

    # normalized_corpus = []  
    # nltk_stop_words = set(stopwords.words("english"))
    # for w in nltk_lemmaList:  
    #     if w not in nltk_stop_words:  
    #         normalized_corpus.append(w)

    # punctuation = ";:.,?!"
    # for word in normalized_corpus:
    #     if word in punctuation:
    #         normalized_corpus.remove(word)

    #still need to add dates and links
    # return normalized_corpus


    
def similiartychecker(tokenA, tokenB):

    A = nlp(tokenA)
    B = nlp(tokenB)
  
    score = A.similarity(B)
  
    return score

# Function to extract skill words from a given corpus.
# ideally, this function will output a set of skills extracted from the corpus
def extractSkills(corpus):

    # run KeyBert Extraction
    keywordsBERT, scoresBERT, avgBERT = extractKeywordsBERT(corpus)

    # run CSO Classifier Extraction
    keywordsCSO = extractKeywordsCSO(corpus)
    
    # extraction method 2
    keywordsYAKE, scoresYAKE, avgYAKE = extractKeywordsYAKE(corpus)

    # extraction method 3
    keywordsTextRank, scoresTextRank, avgTextRank = extractKeywordsTextRank(corpus)

    # return keywordsBERT, keywordsYAKE[0], keywordsTextRank[0]
    # return keywordsBERT, keywordsCSO, keywordsYAKE, keywordsTextRank,
    return {"keywordsBERT": keywordsBERT,
            "keywordsCSO": keywordsCSO,
            "keywordsYAKE": keywordsYAKE}
            # "keywordsTextRAnk": keywordsTextRank}

def extractSkillsforcomp(corpus):

    # run KeyBert Extraction
    keywordsBERT, scoresBERT, avgBERT = extractKeywordsBERT(corpus)

    # run CSO Classifier Extraction
    keywordsCSO = extractKeywordsCSO(corpus)
    
    # extraction method 2
    keywordsYAKE, scoresYAKE, avgYAKE = extractKeywordsYAKE(corpus)

    # extraction method 3
    keywordsTextRank, scoresTextRank, avgTextRank = extractKeywordsTextRank(corpus)

    # return keywordsBERT, keywordsYAKE[0], keywordsTextRank[0]
    return keywordsBERT, keywordsCSO, keywordsYAKE, keywordsTextRank

   
### Driver Code
if __name__ == '__main__':

    # fetch the data
    job_posting_obj = queryJobsAPI("Data Scientist", "Washington, DC")
    resume_dataframe = getFileData('kaggleResumes.csv', 'resumes')
    #---------------

    '''
    # print the dataframes
    print('DataFrame of Job Postings:')
    print(job_posting_dataframe)    
    print()

    print('DataFrame of Resumes:')
    print(resume_dataframe)
    print()
    #----------------------
    '''

    # fetch the job descriptions and resumes by themselves
    jpCorpus = []
    jpTitles = []

    for i in range(len(job_posting_obj)):
      jpCorpus.append(job_posting_obj[i]['MatchedObjectDescriptor']['UserArea']['Details']['JobSummary'])
      jpTitles.append(job_posting_obj[i]['MatchedObjectDescriptor']['PositionTitle'])

    rCorpus  = list(resume_dataframe['Resume'])
    #----------------------

    # # Number of resume samples to view
    # NUM_SAMPLES = 10
    NUM_SAMPLES = 1
    # # number of job postings to view
    # NUM_JPS = 10
    NUM_JPS = 1

    # for each JOB POSTING from the corpus
    i = 0
    all_job_postings = []
    for posting in jpCorpus:
        print('Job Posting #', i+1)
        print()

        text = normalizeCorpus(posting)
        job_posting_json = extractSkills(text)
        all_job_postings.append(text)

        #for similiarity referencer:
        jobKeyBERTsimComp, jobCSOsimComp, jobYAKEsimComp, jobTextRanksimComp = extractSkillsforcomp(text)
       

        # print lines, we are done with this posting
        print('------------------------\n')

        # break, after X postings
        i += 1
        if i > NUM_JPS:
            break
    #---------------------- 

    # for each RESUME from the corpus
    i = 0
    all_resumes_keywords = []
    for resume in rCorpus:
        print('Resume #', i+1)
        print()

        text = normalizeCorpus(resume)
        resume_json = extractSkills(text)
        all_resumes_keywords.append(resume_json)

        #for similiarity referencer:
        resKeyBERTsimComp, resCSOsimComp, resYAKEsimComp, resTextRanksimComp = extractSkillsforcomp(text)

        # print lines, we are done with this resume
        print('------------------------\n')

        # break, after X resumes
        i += 1
        if i > NUM_SAMPLES:
            break
    #---------------------- 
    '''
    ----------------------------------------------------------------------|
    |coss reference similiary of resume and job posting extracted keywords|
    |-addtion by Tyrell Richardson                                        |
    |                                                                     |
    |---------------------------------------------------------------------|
    '''

    keyBERTSimiliarity = np.zeros(14)

    for k in range (0,14):
        keyBERTSimiliarity[k] = similiartychecker(jobKeyBERTsimComp[k], resKeyBERTsimComp[k])

    print('KeyBERT match Similarity Mean: '+str(np.mean(keyBERTSimiliarity)))
    
    #-----------------------------------------------------------------------------

    CSOSimilarity = np.zeros(len(min(jobCSOsimComp, resCSOsimComp)))

    for h in range (0,3):
      CSOSimilarity[h] = similiartychecker(jobCSOsimComp[h],resCSOsimComp[h])

    print('CSO match Similiarity Mean: '+str(np.mean(CSOSimilarity)))

    #-----------------------------------------------------------------------------

    YakeSimilarity = np.zeros(len(min(jobYAKEsimComp, resYAKEsimComp)))
    for i in range (0,19):
      YakeSimilarity[i] = similiartychecker(jobYAKEsimComp[i],resYAKEsimComp[i])

    print('Yake match Similiarity Mean: '+str(np.mean(YakeSimilarity)))

    #-----------------------------------------------------------------------------

    TextRankSimiliarity = np.zeros(5)
    for j in range (0, 5):
      TextRankSimiliarity[j] = similiartychecker(jobTextRanksimComp[j], resTextRanksimComp[j])

    print('TextRank match Similarity Mean: '+str(np.mean(TextRankSimiliarity)))



    
# keep going!

# end of driver code
#---------------

Data Scientist
This position is located in the Department of Health & Human Services (HHS), Centers for Medicare & Medicaid Services (CMS), Office of Burden Reduction and Health Informatics (OBRHI), Emerging Innovations Group . As a Data Scientist, GS-1560-13, you will design, develop, and implement the analytical, statistical, and programming mechanisms necessary to collect, organize, analyze, and interpret unique and highly specialized data sets.
Job Posting # 1

['as', 'a', 'data', 'scientist', 'you', 'will', 'provide', 'data', 'management', 'and', 'analytical', 'support', 'to', 'the', 'technology', 'branch', 'office', 'office', 'of', 'public', 'trust', 'specific', 'to', 'the', 'body', 'worn', 'camera', 'and', 'electronic', 'records', 'management', 'systems', 'this', 'office', 'was', 'created', 'to', 'strengthen', 'public', 'trust', 'in', 'the', 'national', 'park', 'services', 'law', 'enforcement', 'programs', 'through', 'the', 'transparency', 'availability', 'and', 'accessibility',

# Resume and Job Matching

In [91]:
# Resume and Job Posting Matching
matching = SkillsMatching(all_job_postings, jpTitles, all_resumes_keywords)
match_job_postings = matching.skill_match()
matching.print_results(match_job_postings)

------------------------

Matching Job Posting for Resume 1
keywordsBERT
Data Scientist
data scientist provide data management analytical support technology branch office office public trust specific body worn camera electronic records management systems office created strengthen public trust national park services law enforcement programs transparency availability accessibility information fully remote position
keywordsCSO
Data Scientist
position located department health human services hhs centers medicare medicaid services cms office burden reduction health informatics obrhi emerging innovations group data scientist gs design develop implement analytical statistical programming mechanisms necessary collect organize analyze interpret unique highly specialized data sets
keywordsYAKE
Data Scientist
position located department health human services hhs centers medicare medicaid services cms office burden reduction health informatics obrhi emerging innovations group data scientist gs des

# Resume Upload

Drag and Drop a Data Scientist resum into Colab 

Make sure the file is named **resume.pdf**

There is a dummy resume located in the `/data` directory.



< - - - - - - -

In [83]:
!pip install pdfminer3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.0 MB 5.2 MB/s 
     |████████████████████████████████| 2.3 MB 44.2 MB/s 
  Created wheel for pdfminer3: filename=pdfminer3-2018.12.3.0-py3-none-any.whl size=117823 sha256=f066bbfc81d96d284c624077bdbc6e3b56951403c5ccd077575ce3daaecda34b
  Stored in directory: /root/.cache/pip/wheels/1b/bc/f6/b518c318a55ab9e6d72092195cb9b49cac9ca60d6e000d0a1c
Successfully built pdfminer3


In [84]:
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
import sys
from contextlib import redirect_stdout
from io import StringIO 

resource_manager = PDFResourceManager()
fake_file_handle = io.StringIO()
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('resume.pdf', 'rb') as fh:

    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
        page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

# normalize and extract skills 
text = normalizeCorpus(text)
resume_kw = extractSkills(text)

# Matching
matching = SkillsMatching(all_job_postings, jpTitles, [resume_kw])
match_job_postings = matching.skill_match()
matching.print_results(match_job_postings)

FileNotFoundError: ignored